In [1]:
import torch
import os
import matplotlib.pyplot as plt 
import time
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from tqdm.notebook import tqdm
from model import VGG,CNN
import torch.nn as nn
import numpy as np
import random
import cv2
from util import get_file_list,data_split,imgDataset

num_epoch = 500
batch_size=64
embedding_size=4096
epoch_num=100
step_per_epoch=1500
save_model_epoch=30
valid_ratio=0.01

learning_rate={
    0:  0.01,
    40:0.001,
    70:0.0001,
}

result_path='./result'


vgg_face_2_training_face='data/vggface2_train_face_notmask/'
dirs=sorted(os.listdir(vgg_face_2_training_face))
n_classes=len(dirs)


In [2]:
if not os.path.isdir(result_path):
    os.mkdir(result_path)
    
record_path=os.path.join(result_path,'record.csv')
record_fp=open(record_path,'w')
record_fp.write('epoch,train_loss,train_acc,valid_loss,valid_acc\n')


48

In [3]:
image_list,label_list=get_file_list(vgg_face_2_training_face)
train_x,train_y,valid_x,valid_y=data_split(image_list,label_list,valid_ratio=valid_ratio)

In [5]:
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(), # 隨機將圖片水平翻轉
    transforms.ToTensor(), # 將圖片轉成 Tensor，並把數值 normalize 到 [0,1] (data normalization)
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])


valid_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(), 
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])


In [6]:
vgg_face_2_train=imgDataset(file_list=train_x,label_list=train_y,transform=train_transform,image_size=(224,224))
train_dataloader = DataLoader(vgg_face_2_train, batch_size=batch_size, shuffle=True,num_workers=8)

vgg_face_2_valid=imgDataset(file_list=valid_x,label_list=valid_y,transform=train_transform,image_size=(224,224))
valid_dataloader = DataLoader(vgg_face_2_valid, batch_size=batch_size*2, shuffle=True,num_workers=8)


In [7]:
model = CNN(backbone_out_dim=embedding_size,out_dim=n_classes).cuda()
loss = nn.CrossEntropyLoss() 
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate[0],momentum=0.9) 

In [ ]:
min_val_loss=100000000000000000000
for epoch in range(epoch_num):
    
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0
    

    model.train() 
    train_loader_iter=iter(train_dataloader)

    pbar=tqdm(range(step_per_epoch))
    for _ in pbar:
        try:
            batch=next(train_loader_iter)
        except StopIteration:
            train_loader_iter=iter(train_dataloader)
            batch=next(train_loader_iter)
        batch_X,batch_y=batch
        batch_X=batch_X.cuda()
        batch_y=batch_y.cuda()
        optimizer.zero_grad() 
        train_pred = model(batch_X.cuda()) 
        batch_loss = loss(train_pred, batch_y.cuda()) 
        batch_loss.backward() 
        optimizer.step() 

        batch_acc=np.sum(np.argmax(train_pred.cpu().data.numpy(), axis=1) == batch_y.cpu().numpy())/batch_size
        train_acc += batch_acc
        train_loss += batch_loss.item()
        
        pbar.set_description("Epoch {} training:".format(epoch))
        pbar.set_postfix(loss=batch_loss.item(),acc=batch_acc)
        pbar.update()
        
        
    model.eval()
    with torch.no_grad():
        pbar=tqdm(valid_dataloader)
        for batch in pbar:
            batch_X,batch_y=batch
            batch_X=batch_X.cuda()
            batch_y=batch_y.cuda()
            val_pred = model(batch_X.cuda())
            batch_loss = loss(val_pred, batch_y.cuda())
            
            batch_acc=np.sum(np.argmax(val_pred.cpu().data.numpy(), axis=1) == batch_y.cpu().numpy())/(batch_size*2)
            val_acc += batch_acc
            val_loss += batch_loss.item()
            
            pbar.set_description("Epoch {} valid:".format(epoch))
            pbar.set_postfix(loss=batch_loss.item(),acc=batch_acc)
            pbar.update()
    
    train_loss=train_loss/step_per_epoch
    train_acc=train_acc/step_per_epoch
    val_loss=val_loss/len(valid_dataloader)
    val_acc=val_acc/len(valid_dataloader)
    print('Epoch {}|train_loss:{}|train_acc:{}|valid_loss:{}|valid_acc:{}'.format(epoch,train_loss,train_acc,val_loss,val_acc))
    record_fp.write('{},{},{},{},{}\n'.format(epoch,train_loss,train_acc,val_loss,val_acc))
    record_fp.flush()
    if epoch+1 in learning_rate:
        for param_group in optimizer.param_groups:
            param_group['lr'] = learning_rate[epoch+1]
    if val_loss <min_val_loss:
        torch.save(model.state_dict(),os.path.join(result_path,'best_model.pth'))
    if (epoch+1) % save_model_epoch==0:
        torch.save(model.state_dict(),os.path.join(result_path,'model_{}.pth'.format(epoch)))
